### Dataset Import and Inspection

In [104]:
from numpy.ma.extras import average
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# fetch dataset
heart_disease = fetch_ucirepo(id=45)

# data (as pandas dataframes)
X = heart_disease.data.features
y = heart_disease.data.targets

# metadata
# print(heart_disease.metadata)

# variable information
pd.DataFrame(heart_disease.variables)

X = heart_disease.data.features
y = heart_disease.data.targets

# Convert to dataframe
df = pd.concat([X, y], axis=1)

# Inspect data
df.head()
df.info() # ca and thal have null values
df.isna().sum()
(df == '?').sum() # no ? values
(df == 0).sum()

# ca and thal should not have 0 values (but they do)
# ca: number of major vessels (can't be 0)
# thal: thalassemia type (3 is normal, it can't be 0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        299 non-null    float64
 12  thal      301 non-null    float64
 13  num       303 non-null    int64  
dtypes: float64(3), int64(11)
memory usage: 33.3 KB


age           0
sex          97
cp            0
trestbps      0
chol          0
fbs         258
restecg     151
thalach       0
exang       204
oldpeak      99
slope         0
ca          176
thal          0
num         164
dtype: int64

###

### Data Cleanup

In [ ]:
cols_w_invalid_zero = ['ca', 'thal']

# Convert 0's to NaN
for col in cols_w_invalid_zero:
    df[col] = df[col].replace(0,np.nan)

# Check if zeros are gone
(df == 0).sum()

sns.heatmap(df.isna(), cbar=False)
plt.title("Missing Values Heatmap (before cleanup")
plt.show()

# ca and thal have NaN values

# convert zero values of the numerical feature ca to median
median_val = pd.to_numeric(df['ca'], errors='coerce').median()
df['ca'] = df['ca'].fillna(median_val)

# convert zero values of the categorical feature thal to mode (most frequent category)
mode_val = df['thal'].mode()[0]
df['thal'] = df['thal'].fillna(median_val)

# Check if there are missing values after cleanup
sns.heatmap(df.isna(), cbar=False)
plt.title("Missing Values Heatmap (after cleanup)")
plt.show()



### Check for class imbalance

In [ ]:

# Count target classes
class_counts = df['num'].value_counts().sort_index()

plt.figure(figsize=(6,4))
sns.barplot(
    x=class_counts.index,
    y=class_counts.values
)

plt.xlabel('Target Class', labelpad=20)
plt.ylabel('Number of Samples', labelpad=20)
plt.title('Class Distribution (Heart Disease)')
plt.xticks(rotation=30, ha='right')
plt.xticks([0, 1, 2, 3, 4], ['No Heart Disease (0)', 'Mild Heart Disease (1)', 'Moderate Heart Disease (2)', 'Severe Heart Disease (3)', 'Very Severe Heart Disease (4)'])

plt.show()

# Print counts and proportions
print("Class counts:\n", class_counts)
print("\nClass proportions:\n", class_counts / class_counts.sum())

# plot shows that small class imbalance with slightly
# more no heart disease observations than those with heart disease
#
# 164 people don't have heart disease
# 55 people have mild heart disease
# 36 people have moderate heart disease
# 35 people have severe heart disease
# 13 people have very severe heart disease

### EDA (Exploratory Data Analysis)

#### Numerical Feature Distribution

In [ ]:
numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

df[numerical_cols].hist(figsize=(12,8), bins=20)
plt.suptitle("Distributions of Numerical Features")
plt.tight_layout()
plt.show()

#### Outlier Detection

In [ ]:
plt.figure(figsize=(12,6))
df[numerical_cols].boxplot()
plt.title("Boxplots of Numerical Features")
plt.ylabel("Value")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### Counts of Categorical Features

In [ ]:
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal', 'ca']

for col in categorical_cols:
    plt.figure(figsize=(5,3))
    sns.countplot(x=col, data=df)
    plt.title(f'Distribution of {col}')
    plt.tight_layout()
    plt.show()

### Numerical Features vs Target

In [ ]:
for col in ['age', 'chol', 'thalach']:
    plt.figure(figsize=(5,3))
    sns.kdeplot(data=df, x=col, hue='num', fill=True)
    plt.title(f'{col} by Heart Disease Status')
    plt.tight_layout()
    plt.show()

#### Categorical Features vs Target

In [ ]:
for col in ['cp', 'exang', 'thal', 'ca']:
    plt.figure(figsize=(5,3))
    sns.countplot(x=col, hue='num', data=df)
    plt.title(f'{col} vs num')
    plt.tight_layout()
    plt.show()


#### Correlation Matrix and Heatmap

In [ ]:
corr = df[numerical_cols + ['num']].corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.tight_layout()
plt.show()

In [ ]:
#### Pairplot

In [ ]:
sns.pairplot(
    df[['age', 'thalach', 'chol', 'oldpeak', 'num']],
    hue='num'
)
plt.show()

### T-Test

In [ ]:
group0 = df[df['num'] == 0]
group1 = df[df['num'] == 1]

for col in numerical_cols:
    stat, p = ttest_ind(group0[col], group1[col])
    print(f"{col}: p-value = {p:.4f}")

#### Chi-square Test

In [ ]:
contingency = pd.crosstab(df['cp'], df['num'])
chi2, p, dof, exp = chi2_contingency(contingency)
print("Chest Pain vs Target p-value:", p)

### Modeling

#### Evaluate Function

In [95]:
# Function to evaluate models in later sections
# It will be reused to determine accuracy, precision,
# recall, and ROC AUC

def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)

    return {
        'accuracy_score': accuracy_score(y_test, y_pred),
        'precision_score': precision_score(y_test, y_pred, average='weighted'),
        'recall_score': recall_score(y_test, y_pred, average='weighted'),
        'roc_auc_score': roc_auc_score(y_test, y_proba, multi_class='ovr', average='weighted')
    }

#### Train / Test Split

In [74]:
# Split data into features and target
X = df.drop(columns=['num'], axis=1)
y = df['num']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# Split X and y to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#### Pre-processing Pipeline

In [75]:

preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', StandardScaler(), numerical_cols),
        ('categorical', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ]
)

#### Model 1 - Logistic Regression

In [105]:
from sklearn.linear_model import LogisticRegression

lr = Pipeline([
    ('preprocess', preprocessor),
    ('model', LogisticRegression(
        max_iter=1000,
        class_weight='balanced'
    ))
])

lr.fit(X_train, y_train)

lr_scores = evaluate(lr, X_test, y_test)
lr_scores


{'accuracy_score': 0.45901639344262296,
 'precision_score': 0.4597775175644028,
 'recall_score': 0.45901639344262296,
 'roc_auc_score': 0.775296286501297}

#### Model 2 - Random Forest

In [98]:
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline([
    ('preprocess', preprocessor),
    ('model', RandomForestClassifier(
        n_estimators=200,
        class_weight='balanced',
        random_state=42
    ))
])

rf.fit(X_train, y_train)

rf_scores = evaluate(lr, X_test, y_test)
rf_scores

{'accuracy_score': 0.45901639344262296,
 'precision_score': 0.4597775175644028,
 'recall_score': 0.45901639344262296,
 'roc_auc_score': 0.775296286501297}

#### Model 3 - Support Vector Machine

In [103]:
from sklearn.svm import SVC

svm = Pipeline([
    ('preprocess', preprocessor),
    ('model', SVC(
        kernel='rbf',
        probability=True,
        class_weight='balanced'
    ))
])

svm.fit(X_train, y_train)

svm_scores = evaluate(svm, X_test, y_test)
svm_scores

{'accuracy_score': 0.4262295081967213,
 'precision_score': 0.47372654667736636,
 'recall_score': 0.4262295081967213,
 'roc_auc_score': 0.8119545919257802}

#### Model 4 - Gradient Boosting

In [106]:
from sklearn.ensemble import GradientBoostingClassifier

gb = Pipeline([
    ('preprocess', preprocessor),
    ('model', GradientBoostingClassifier(random_state=42))
])

gb.fit(X_train, y_train)
gb_scores = evaluate(svm, X_test, y_test)
gb_scores

{'accuracy_score': 0.4262295081967213,
 'precision_score': 0.47372654667736636,
 'recall_score': 0.4262295081967213,
 'roc_auc_score': 0.8119545919257802}

#### Model 5 - KNN

In [107]:
from sklearn.neighbors import KNeighborsClassifier

knn = Pipeline([
    ('preprocess', preprocessor),
    ('model', KNeighborsClassifier(n_neighbors=7))
])

knn.fit(X_train, y_train)

knn_scores = evaluate(knn, X_test, y_test)
knn_scores

{'accuracy_score': 0.5245901639344263,
 'precision_score': 0.48629881380781026,
 'recall_score': 0.5245901639344263,
 'roc_auc_score': 0.7888288940477558}

#### Compare All 5 Models Above

In [109]:
models = {
    'Logistic Regression': lr,
    'Random Forest': rf,
    'SVM': svm,
    'Gradient Boosting': gb,
    'KNN': knn
}

results = pd.DataFrame({
    name: evaluate(model, X_test, y_test)
    for name, model in models.items()
}).T

results

,accuracy_score,precision_score,recall_score,roc_auc_score
Logistic Regression,0.459016,0.459778,0.459016,0.775296
Random Forest,0.540984,0.483308,0.540984,0.812433
SVM,0.426230,0.473727,0.426230,0.811955
Gradient Boosting,0.475410,0.407228,0.475410,0.760389
KNN,0.524590,0.486299,0.524590,0.788829


#### Hyperparameter Tuning

In [117]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'model__C': [0.01, 0.1, 1, 10],
    'model__penalty': ['l1', 'l2'],
    'model__solver': ['liblinear']
}

grid_lr = GridSearchCV(
    lr,
    param_grid,
    cv=5,
    scoring='recall',
    n_jobs=-1
)

grid_lr.fit(X_train, y_train)

,estimator,Pipeline(step..._iter=1000))])
,param_grid,"{'model__C': [0.01, 0.1, ...], 'model__penalty': ['l1', 'l2'], 'model__solver': ['liblinear']}"
,scoring,'recall'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('numerical', ...), ('categorical', ...)]"


#### Evaluate All 5 Models Above